In [1]:


!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 110.5 MB/s eta 0:00:00
  Attempting uninstall: cachetools
    Found existing installation: cachetools 7.0.0
    Uninstalling cachetools-7.0.0:
      Successfully uninstalled cachetools-7.0.0


In [2]:
!pip install gradio -q

In [3]:
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from google.colab import drive
import gradio as gr
from openai import OpenAI


In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_csv("/content/drive/MyDrive/master_logistic_audit.csv")

In [6]:
client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key="GROQ_API_KEY"  #INSERT YOUR GROQ API KEY HERE
)

In [7]:
def get_llm_recommendation(state, original_category):
    risk = "high" if state in ['AL', 'MA', 'SE', 'PI', 'CE', 'RR'] else "medium/low"

    prompt = f"""
You are a helpful shopping assistant for Olist (Veridi Logistics).
Customer is in {state} state, originally interested in: "{original_category}".

Delivery risk for some categories in this region is {risk}.

Suggest **3 alternative English category names** that are:
- Likely to be delivered faster/more reliably
- Have higher average customer satisfaction
- Relevant or complementary to the original item

For each suggestion include one short reason.
Keep response friendly, concise, under 120 words.
Use real Olist English categories (e.g. health_beauty, perfumery, electronics, toys, watches_gifts, bed_bath_table).
"""

    try:
        response = client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=220,
            temperature=0.7
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"⚠️ LLM Error: {str(e)}\n(Check API key or try again later)"

In [9]:
def show_dashboard(selected_state):
    if selected_state == "All":
        filtered = df
    else:
        filtered = df[df['customer_state'] == selected_state]

    # Late percentage
    delivered = filtered[filtered['Delivery_Status'] != 'Not Delivered']
    if len(delivered) == 0:
        late_pct = 0.0
        state_df = pd.DataFrame()
        review_df = pd.DataFrame()
    else:
        late_pct = (
            delivered[delivered['Delivery_Status'].isin(['Late', 'Super Late'])].shape[0]
            / len(delivered) * 100
        )

        # State-level late % (top 12)
        state_late = (
            delivered.groupby('customer_state')['Delivery_Status']
            .apply(lambda x: (x.isin(['Late', 'Super Late'])).mean() * 100)
            .reset_index(name='% Late')
            .sort_values('% Late', ascending=False)
            .head(12)
        )

        # Review by status
        review_by_status = (
            delivered.groupby('Delivery_Status')['review_score']
            .mean()
            .reset_index()
            .sort_values('review_score', ascending=False)
        )

        fig_state = px.bar(
            state_late,
            x='customer_state',
            y='% Late',
            title=f"Late / Super Late % by State (filtered: {selected_state})",
            color='% Late',
            color_continuous_scale='RdYlGn_r'
        )
        fig_state.update_layout(xaxis_title="State", yaxis_title="% Late", height=450)

        fig_review = px.bar(
            review_by_status,
            x='Delivery_Status',
            y='review_score',
            title="Average Review Score by Delivery Status",
            color='Delivery_Status',
            color_discrete_sequence=['#2ecc71', '#f1c40f', '#e74c3c', '#95a5a6']
        )
        fig_review.update_layout(yaxis_range=[0,5], height=450)

    summary_text = f"**Overall Late / Super Late**: {late_pct:.1f}%\n\n**Average Review Score**: {filtered['review_score'].mean():.2f}/5"

    return summary_text, fig_state, fig_review




In [10]:
#Gradio Interface
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 📦 Olist Logistics Auditor Dashboard
    Explore delivery performance and get smart product recommendations
    """)

    with gr.Row():
        state_dropdown = gr.Dropdown(
            choices=["All"] + sorted(df['customer_state'].unique().tolist()),
            label="Filter by Customer State",
            value="All",
            interactive=True
        )

    summary_output = gr.Markdown(label="Summary")
    plot_state = gr.Plot()
    plot_review = gr.Plot()

    # LLM Recommendation Section
    gr.Markdown("### 🧠 Smart Product Recommendations")
    gr.Markdown("Avoid delays — get 3 safer alternative categories")

    with gr.Row():
        rec_state = gr.Dropdown(
            choices=sorted(df['customer_state'].unique().tolist()),
            label="Customer State",
            value="SP"
        )
        original_cat = gr.Textbox(
            label="Original Category (English)",
            placeholder="e.g. bed_bath_table, moveis_escritorio, electronics",
            value="bed_bath_table"
        )

    recommend_button = gr.Button("Get Recommendations", variant="primary")
    recommendation_output = gr.Markdown()

    # Wire up events
    state_dropdown.change(
        fn=show_dashboard,
        inputs=state_dropdown,
        outputs=[summary_output, plot_state, plot_review]
    )

    recommend_button.click(
        fn=get_llm_recommendation,
        inputs=[rec_state, original_cat],
        outputs=recommendation_output
    )

    # Initial load
    demo.load(show_dashboard, inputs=state_dropdown, outputs=[summary_output, plot_state, plot_review])

demo.launch(share=True, debug=False)

/tmp/ipython-input-1193875254.py:2: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1ba64bf49a4fb06bc8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
